## Random forest guided example drill 

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 300)# our data set has too many rows 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn import tree
from sklearn import ensemble
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
%matplotlib inline


In [2]:
y2015 = pd.read_csv('LoanStats3d.csv', skipinitialspace=True,
    header=1)
y2015.head()

C:\Users\Omistaja\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,68009401,72868139.0,16000.0,16000.0,16000.0,60 months,14.85%,379.39,C,C5,Bookkeeper/Accounting,10+ years,MORTGAGE,48000.0,Not Verified,Dec-2015,Current,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,credit_card,Credit card refinancing,297xx,SC,33.18,0.0,Jun-1991,0.0,33.0,2.0,11.0,2.0,19108.0,29.6%,19.0,w,13668.88,13668.88,4519.68,4519.68,2331.12,2188.56,0.0,0.0,0.0,Jan-2017,379.39,Jan-2017,Jan-2017,0.0,33.0,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,0.0,31329.0,1.0,2.0,1.0,2.0,6.0,12221.0,91.0,1.0,4.0,14982.0,65.0,284700.0,0.0,0.0,1.0,6.0,2848.0,263953.0,17.6,0.0,0.0,131.0,294.0,11.0,6.0,2.0,11.0,76.0,11.0,76.0,2.0,6.0,9.0,6.0,8.0,6.0,9.0,11.0,9.0,11.0,0.0,0.0,0.0,2.0,78.9,0.0,0.0,2.0,298100.0,31329.0,281300.0,13400.0
1,68354783,73244544.0,9600.0,9600.0,9600.0,36 months,7.49%,298.58,A,A4,tech,8 years,MORTGAGE,60000.0,Not Verified,Dec-2015,Current,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,credit_card,Credit card refinancing,299xx,SC,22.44,0.0,Jun-1996,0.0,NaN,NaN,7.0,0.0,7722.0,59.4%,9.0,w,6635.69,6635.69,3572.97,3572.97,2964.31,608.66,0.0,0.0,0.0,Jan-2017,298.58,Jan-2017,Jan-2017,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,0.0,55387.0,0.0,3.0,0.0,0.0,47.0,47665.0,43.0,2.0,2.0,4744.0,50.0,13000.0,0.0,0.0,2.0,2.0,7912.0,4778.0,61.8,0.0,0.0,234.0,91.0,9.0,9.0,0.0,11.0,NaN,9.0,NaN,0.0,3.0,3.0,3.0,3.0,5.0,4.0,4.0,3.0,7.0,0.0,0.0,0.0,2.0,100.0,66.7,0.0,0.0,88635.0,55387.0,12500.0,75635.0
2,68466916,73356753.0,25000.0,25000.0,25000.0,36 months,7.49%,777.55,A,A4,Sales Manager,10+ years,MORTGAGE,109000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,226xx,VA,26.02,0.0,Dec-2001,1.0,NaN,NaN,9.0,0.0,20862.0,54.3%,19.0,w,0.00,0.00,26224.23,26224.23,25000.00,1224.23,0.0,0.0,0.0,Sep-2016,20807.39,NaN,Jan-2017,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,0.0,305781.0,0.0,3.0,0.0,1.0,13.0,47194.0,58.0,0.0,1.0,8937.0,57.0,38400.0,1.0,0.0,1.0,2.0,33976.0,17538.0,54.3,0.0,0.0,142.0,168.0,13.0,13.0,3.0,13.0,NaN,0.0,NaN,0.0,3.0,3.0,5.0,6.0,7.0,5.0,9.0,3.0,9.0,0.0,0.0,0.0,0.0,100.0,20.0,0.0,0.0,373572.0,68056.0,38400.0,82117.0
3,68466961,73356799.0,28000.0,28000.0,28000.0,36 months,6.49%,858.05,A,A2,Senior Manager,10+ years,MORTGAGE,92000.0,Not Verified,Dec-2015,Current,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,275xx,NC,21.60,0.0,May-1984,0.0,42.0,NaN,16.0,0.0,51507.0,6

In [ ]:
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')


In [3]:
# Drop other columns with many unique variables
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc'], 1, inplace=True)

In [4]:
# Remove two summary rows at the end that don't actually contain data.
y2015 = y2015[:-2]

In [5]:
y2015.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,68009401,72868139.0,16000.0,16000.0,16000.0,60 months,14.85%,379.39,C,10+ years,MORTGAGE,48000.0,Not Verified,Dec-2015,Current,n,credit_card,Credit card refinancing,33.18,0.0,0.0,33.0,2.0,11.0,2.0,19108.0,19.0,w,13668.88,13668.88,4519.68,4519.68,2331.12,2188.56,0.0,0.0,0.0,Jan-2017,379.39,Jan-2017,Jan-2017,0.0,33.0,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,0.0,31329.0,1.0,2.0,1.0,2.0,6.0,12221.0,91.0,1.0,4.0,14982.0,65.0,284700.0,0.0,0.0,1.0,6.0,2848.0,263953.0,17.6,0.0,0.0,131.0,294.0,11.0,6.0,2.0,11.0,76.0,11.0,76.0,2.0,6.0,9.0,6.0,8.0,6.0,9.0,11.0,9.0,11.0,0.0,0.0,0.0,2.0,78.9,0.0,0.0,2.0,298100.0,31329.0,281300.0,13400.0
1,68354783,73244544.0,9600.0,9600.0,9600.0,36 months,7.49%,298.58,A,8 years,MORTGAGE,60000.0,Not Verified,Dec-2015,Current,n,credit_card,Credit card refinancing,22.44,0.0,0.0,NaN,NaN,7.0,0.0,7722.0,9.0,w,6635.69,6635.69,3572.97,3572.97,2964.31,608.66,0.0,0.0,0.0,Jan-2017,298.58,Jan-2017,Jan-2017,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,0.0,55387.0,0.0,3.0,0.0,0.0,47.0,47665.0,43.0,2.0,2.0,4744.0,50.0,13000.0,0.0,0.0,2.0,2.0,7912.0,4778.0,61.8,0.0,0.0,234.0,91.0,9.0,9.0,0.0,11.0,NaN,9.0,NaN,0.0,3.0,3.0,3.0,3.0,5.0,4.0,4.0,3.0,7.0,0.0,0.0,0.0,2.0,100.0,66.7,0.0,0.0,88635.0,55387.0,12500.0,75635.0
2,68466916,73356753.0,25000.0,25000.0,25000.0,36 months,7.49%,777.55,A,10+ years,MORTGAGE,109000.0,Not Verified,Dec-2015,Fully Paid,n,debt_consolidation,Debt consolidation,26.02,0.0,1.0,NaN,NaN,9.0,0.0,20862.0,19.0,w,0.00,0.00,26224.23,26224.23,25000.00,1224.23,0.0,0.0,0.0,Sep-2016,20807.39,NaN,Jan-2017,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,0.0,305781.0,0.0,3.0,0.0,1.0,13.0,47194.0,58.0,0.0,1.0,8937.0,57.0,38400.0,1.0,0.0,1.0,2.0,33976.0,17538.0,54.3,0.0,0.0,142.0,168.0,13.0,13.0,3.0,13.0,NaN,0.0,NaN,0.0,3.0,3.0,5.0,6.0,7.0,5.0,9.0,3.0,9.0,0.0,0.0,0.0,0.0,100.0,20.0,0.0,0.0,373572.0,68056.0,38400.0,82117.0
3,68466961,73356799.0,28000.0,28000.0,28000.0,36 months,6.49%,858.05,A,10+ years,MORTGAGE,92000.0,Not Verified,Dec-2015,Current,n,debt_consolidation,Debt consolidation,21.60,0.0,0.0,42.0,NaN,16.0,0.0,51507.0,24.0,w,19263.77,19263.77,10271.36,10271.36,8736.23,1535.13,0.0,0.0,0.0,Jan-2017,858.05,Jan-2017,Jan-2017,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,0.0,221110.0,0.0,2.0,0.0,0.0,27.0,23413.0,55.0,0.0,1.0,8752.0,61.0,79900.0,0.0,1.0,0.0,1.0,13819.0,16623.0,59.9,0.0,0.0,82.0,379.0,19.0,19.0,2.0,19.0,48.0,NaN,42.0,0.0,7.0,9.0,9.0,11.0,4.0,13.0,18.0,9.0,16.0,0.0,0.0,0.0,0.0,91.7,22.2,0.0,0.0,304003.0,74920.0,41500.0,42503.0
4,68495092,73384866.0,8650.0,8650

### DRILL: Third Attempt
So here's your task. Get rid of as much data as possible without dropping below an average of 90% accuracy in a 10-fold cross validation.

You'll want to do a few things in this process. First, dive into the data that we have and see which features are most important. This can be the raw features or the generated dummies. You may want to use PCA or correlation matrices.

Can you do it without using anything related to payment amount or outstanding principal? How do you know?

In [8]:
# Selecting variables by judging what they reprent and their importance to the outcome.
# too many to do heatmap 
features = y2015[['loan_amnt', 'int_rate', 'term', 'installment', 'emp_length', 'annual_inc', 'title', 'revol_bal', 'total_pymnt',
                  'total_rec_prncp', 'last_pymnt_amnt', 'application_type', 'tot_cur_bal', 'mths_since_rcnt_il', 'max_bal_bc',
                 'all_util', 'avg_cur_bal', 'mo_sin_rcnt_tl', 'mort_acc', 'num_il_tl', 'num_sats', 'total_il_high_credit_limit', 'loan_status']]

In [19]:
features.describe()

,loan_amnt,installment,annual_inc,revol_bal,total_pymnt,total_rec_prncp,last_pymnt_amnt,tot_cur_bal,mths_since_rcnt_il,max_bal_bc,all_util,avg_cur_bal,mo_sin_rcnt_tl,mort_acc,num_il_tl,num_sats,total_il_high_credit_limit
count,421095.000000,421095.000000,4.210950e+05,4.210950e+05,421095.000000,421095.000000,421095.000000,4.210950e+05,20810.000000,21372.000000,21372.000000,421095.000000,421095.000000,421095.000000,421095.000000,421095.000000,4.210950e+05
mean,15240.285862,441.860087,7.696561e+04,1.768548e+04,9272.455831,7098.480969,2641.275728,1.400755e+05,20.912686,5887.979740,60.881995,13104.784721,8.013204,1.685261,8.649001,11.894537,4.347865e+04
std,8571.325901,244.851179,7.394996e+04,2.419503e+04,6978.424974,6376.095419,5642.581267,1.552696e+05,27.209081,5284.701239,20.013842,15671.803897,9.189312,1.990160,7.471041,5.615119,4.394271e+04
min,1000.000000,23.400000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
25%,8500.000000,263.930000,4.600000e+04,6.480000e+03,4565.115000,3000.000000,295.375000,3.103900e+04,6.000000,2411.000000,48.000000,3199.000000,3.000000,0.000000,4.000000,8.000000,1.548150e+04
50%,14000.000000,385.410000,6.500000e+04,1.199600e+04,7323.930000,5000.000000,474.650000,7.972900e+04,12.000000,4483.000000,62.000000,7262.000000,5.000000,1.000000,7.000000,11.000000,3.297000e+04
75%,20000.000000,578.790000,9.169050e+04,2.142800e+04,11781.365000,8866.475000,863.160000,2.074750e+05,23.000000,7771.500000,75.000000,17974.500000,10.000000,3.000000,11.000000,15.000000,5.829900e+04
max,35000.000000,1445.460000,9.500000e+06,2.904836e+06,50312.207800,35000.000000,36475.590000,4.447397e+06,363.000000,127305.000000,151.000000,555925.000000,263.000000,52.000000,132.000000,90.000000,2.101913e+06


In [9]:
pd.get_dummies(features).head()

,loan_amnt,installment,annual_inc,revol_bal,total_pymnt,total_rec_prncp,last_pymnt_amnt,tot_cur_bal,mths_since_rcnt_il,max_bal_bc,all_util,avg_cur_bal,mo_sin_rcnt_tl,mort_acc,num_il_tl,num_sats,total_il_high_credit_limit,int_rate_ 5.32%,int_rate_ 5.93%,int_rate_ 6.00%,int_rate_ 6.03%,int_rate_ 6.24%,int_rate_ 6.39%,int_rate_ 6.49%,int_rate_ 6.68%,int_rate_ 6.89%,int_rate_ 6.92%,int_rate_ 6.99%,int_rate_ 7.26%,int_rate_ 7.49%,int_rate_ 7.89%,int_rate_ 7.91%,int_rate_ 8.18%,int_rate_ 8.19%,int_rate_ 8.38%,int_rate_ 8.49%,int_rate_ 8.67%,int_rate_ 9.17%,int_rate_ 9.49%,int_rate_ 9.76%,int_rate_ 9.80%,int_rate_ 9.99%,int_rate_ 10.49%,int_rate_ 10.64%,int_rate_ 10.78%,int_rate_ 10.99%,int_rate_ 11.22%,int_rate_ 11.44%,int_rate_ 11.48%,int_rate_ 11.49%,int_rate_ 11.53%,int_rate_ 11.99%,int_rate_ 12.05%,int_rate_ 12.29%,int_rate_ 12.39%,int_rate_ 12.59%,int_rate_ 12.69%,int_rate_ 12.88%,int_rate_ 12.99%,int_rate_ 13.18%,int_rate_ 13.33%,int_rate_ 13.44%,int_rate_ 13.66%,int_rate_ 13.67%,int_rate_ 13.99%,int_rate_ 14.31%,int_rate_ 14.33%,int_rate_ 14.48%,int_rate_ 14.65%,int_rate_ 14.85%,int_rate_ 14.99%,int_rate_ 15.41%,int_rate_ 15.59%,int_rate_ 15.61%,int_rate_ 15.77%,int_rate_ 15.99%,int_rate_ 16.49%,int_rate_ 16.55%,int_rate_ 16.59%,int_rate_ 16.99%,int_rate_ 17.14%,int_rate_ 17.27%,int_rate_ 17.57%,int_rate_ 17.86%,int_rate_ 17.97%,int_rate_ 18.20%,int_rate_ 18.25%,int_rate_ 18.49%,int_rate_ 18.54%,int_rate_ 18.55%,int_rate_ 18.84%,int_rate_ 18.99%,int_rate_ 19.19%,int_rate_ 19.24%,int_rate_ 19.48%,int_rate_ 19.52%,int_rate_ 19.89%,int_rate_ 19.99%,int_rate_ 20.49%,int_rate_ 20.50%,int_rate_ 20.99%,int_rate_ 21.48%,int_rate_ 21.67%,int_rate_ 21.99%,int_rate_ 22.45%,int_rate_ 22.99%,int_rate_ 23.13%,int_rate_ 23.99%,int_rate_ 24.24%,int_rate_ 24.50%,int_rate_ 24.99%,int_rate_ 25.09%,int_rate_ 25.57%,int_rate_ 25.78%,int_rate_ 25.80%,int_rate_ 25.83%,int_rate_ 25.89%,int_rate_ 25.99%,int_rate_ 26.06%,int_rate_ 26.77%,int_rate_ 26.99%,int_rate_ 27.31%,int_rate_ 27.49%,int_rate_ 27.88%,int_rate_ 27.99%,int_rate_ 28.49%,int_rate_ 28.99%,term_ 36 months,term_ 60 months,emp_length_1 year,emp_length_10+ years,emp_length_2 years,emp_length_3 years,emp_length_4 years,emp_length_5 years,emp_length_6 years,emp_length_7 years,emp_length_8 years,emp_length_9 years,emp_length_< 1 year,title_Business,title_Car financing,title_Credit Card/Auto Repair,title_Credit card refinancing,title_Debt consolidation,title_DebtC,title_Green loan,title_Home buying,title_Home improvement,title_Learning and training,title_Major purchase,title_Medical expenses,title_Moving and relocation,title_New Baby and New House (CC Consolidate),title_Other,title_Pay off Lowes Card,title_Paying off higher interest cards & auto,title_Prescription Drug and Medical Costs,title_SAVE,title_Simple Loan Until Contract Is Completed,title_Student Loan,title_Trying to come back to reality!,title_Vacation,title_considerate,title_new day,title_new kitchen for momma!,title_odymeds,application_type_INDIVIDUAL,application_type_JOINT,loan_status_Charged Off,loan_status_Current,loan_status_Default,loan_status_Fully Paid,loan_status_In Grace Period,loan_status_Late (16-30 days),loan_status_Late (31-120 days)
0,16000.0,379.39,48000.0,19108.0,4519.68,2331.12,379.39,31329.0,6.0,14982.0,65.0,2848.0,6.0,2.0,6.0,11.0,13400.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
1,9600.0,298.58,60000.0,7722.0,3572.97,2964.31,298.58,55387.0,47.0,4744.0,50.0,7912.0,9.0,0.0,5.0,7.0,75635.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,

In [10]:
rfc = ensemble.RandomForestClassifier()
X = features.drop('loan_status', 1)
Y = features['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10)


array([0.90289948, 0.94666477, 0.9423666 , 0.93479138, 0.92835431,
       0.92702446, 0.92464794, 0.92355553, 0.91913459, 0.76604284])

### Without using anything related to payment amount or outstanding principal

In [21]:
# features without payment amount, outstanding principal, or similar 
features_w = y2015[['loan_amnt', 'int_rate', 'term', 'emp_length', 'annual_inc', 'title', 'revol_bal', 'application_type', 'mths_since_rcnt_il', 'max_bal_bc',
                 'all_util', 'avg_cur_bal', 'mo_sin_rcnt_tl', 'mort_acc', 'num_il_tl', 'num_sats', 'total_il_high_credit_limit', 'loan_status']]

In [22]:
pd.get_dummies(features_w).head()

,loan_amnt,annual_inc,revol_bal,mths_since_rcnt_il,max_bal_bc,all_util,avg_cur_bal,mo_sin_rcnt_tl,mort_acc,num_il_tl,num_sats,total_il_high_credit_limit,int_rate_ 5.32%,int_rate_ 5.93%,int_rate_ 6.00%,int_rate_ 6.03%,int_rate_ 6.24%,int_rate_ 6.39%,int_rate_ 6.49%,int_rate_ 6.68%,int_rate_ 6.89%,int_rate_ 6.92%,int_rate_ 6.99%,int_rate_ 7.26%,int_rate_ 7.49%,int_rate_ 7.89%,int_rate_ 7.91%,int_rate_ 8.18%,int_rate_ 8.19%,int_rate_ 8.38%,int_rate_ 8.49%,int_rate_ 8.67%,int_rate_ 9.17%,int_rate_ 9.49%,int_rate_ 9.76%,int_rate_ 9.80%,int_rate_ 9.99%,int_rate_ 10.49%,int_rate_ 10.64%,int_rate_ 10.78%,int_rate_ 10.99%,int_rate_ 11.22%,int_rate_ 11.44%,int_rate_ 11.48%,int_rate_ 11.49%,int_rate_ 11.53%,int_rate_ 11.99%,int_rate_ 12.05%,int_rate_ 12.29%,int_rate_ 12.39%,int_rate_ 12.59%,int_rate_ 12.69%,int_rate_ 12.88%,int_rate_ 12.99%,int_rate_ 13.18%,int_rate_ 13.33%,int_rate_ 13.44%,int_rate_ 13.66%,int_rate_ 13.67%,int_rate_ 13.99%,int_rate_ 14.31%,int_rate_ 14.33%,int_rate_ 14.48%,int_rate_ 14.65%,int_rate_ 14.85%,int_rate_ 14.99%,int_rate_ 15.41%,int_rate_ 15.59%,int_rate_ 15.61%,int_rate_ 15.77%,int_rate_ 15.99%,int_rate_ 16.49%,int_rate_ 16.55%,int_rate_ 16.59%,int_rate_ 16.99%,int_rate_ 17.14%,int_rate_ 17.27%,int_rate_ 17.57%,int_rate_ 17.86%,int_rate_ 17.97%,int_rate_ 18.20%,int_rate_ 18.25%,int_rate_ 18.49%,int_rate_ 18.54%,int_rate_ 18.55%,int_rate_ 18.84%,int_rate_ 18.99%,int_rate_ 19.19%,int_rate_ 19.24%,int_rate_ 19.48%,int_rate_ 19.52%,int_rate_ 19.89%,int_rate_ 19.99%,int_rate_ 20.49%,int_rate_ 20.50%,int_rate_ 20.99%,int_rate_ 21.48%,int_rate_ 21.67%,int_rate_ 21.99%,int_rate_ 22.45%,int_rate_ 22.99%,int_rate_ 23.13%,int_rate_ 23.99%,int_rate_ 24.24%,int_rate_ 24.50%,int_rate_ 24.99%,int_rate_ 25.09%,int_rate_ 25.57%,int_rate_ 25.78%,int_rate_ 25.80%,int_rate_ 25.83%,int_rate_ 25.89%,int_rate_ 25.99%,int_rate_ 26.06%,int_rate_ 26.77%,int_rate_ 26.99%,int_rate_ 27.31%,int_rate_ 27.49%,int_rate_ 27.88%,int_rate_ 27.99%,int_rate_ 28.49%,int_rate_ 28.99%,term_ 36 months,term_ 60 months,emp_length_1 year,emp_length_10+ years,emp_length_2 years,emp_length_3 years,emp_length_4 years,emp_length_5 years,emp_length_6 years,emp_length_7 years,emp_length_8 years,emp_length_9 years,emp_length_< 1 year,title_Business,title_Car financing,title_Credit Card/Auto Repair,title_Credit card refinancing,title_Debt consolidation,title_DebtC,title_Green loan,title_Home buying,title_Home improvement,title_Learning and training,title_Major purchase,title_Medical expenses,title_Moving and relocation,title_New Baby and New House (CC Consolidate),title_Other,title_Pay off Lowes Card,title_Paying off higher interest cards & auto,title_Prescription Drug and Medical Costs,title_SAVE,title_Simple Loan Until Contract Is Completed,title_Student Loan,title_Trying to come back to reality!,title_Vacation,title_considerate,title_new day,title_new kitchen for momma!,title_odymeds,application_type_INDIVIDUAL,application_type_JOINT,loan_status_Charged Off,loan_status_Current,loan_status_Default,loan_status_Fully Paid,loan_status_In Grace Period,loan_status_Late (16-30 days),loan_status_Late (31-120 days)
0,16000.0,48000.0,19108.0,6.0,14982.0,65.0,2848.0,6.0,2.0,6.0,11.0,13400.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
1,9600.0,60000.0,7722.0,47.0,4744.0,50.0,7912.0,9.0,0.0,5.0,7.0,75635.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
2,25000.0,109000.0,20862.0,13.0,8937.0,57.0,33976.0,13.0,3.0,7.0,9.0,82117.0,0,0,0,0

In [23]:
rfc = ensemble.RandomForestClassifier()
X_w = features_w.drop('loan_status', 1)
Y_w = features_w['loan_status']
X_w = pd.get_dummies(X_w)
X_w = X_w.dropna(axis=1)

cross_val_score(rfc, X_w, Y_w, cv=10)

array([0.6634846 , 0.65586189, 0.66376956, 0.6631284 , 0.66449774,
       0.66217051, 0.6591465 , 0.65703294, 0.64746954, 0.40775661])

It looks like features related to payment amount or outstanding principal carry a lot of information. As can be seen in the cross validation score above, the accuracy drops significantly. 

### Dimentionality reduction using Pca 

Performing a principal component analysis on this mic of categorical and continious variables resulted in a poor result. The first 100 components explain only 67% of the varience

In [11]:
X_nor = StandardScaler().fit_transform(X)

In [16]:
pca = PCA(n_components=100)

In [17]:
X_pca = pca.fit_transform(X_nor)
varience = pca.explained_variance_ratio_.cumsum()
print(varience)

[0.02861304 0.04323255 0.05696608 0.06902784 0.07992656 0.09001042
 0.09791465 0.10578674 0.11247204 0.11902499 0.12556403 0.13204798
 0.1384704  0.14487918 0.15127138 0.15762492 0.16395287 0.17026378
 0.17654923 0.1828301  0.18910221 0.19536013 0.20160694 0.20784502
 0.21407678 0.22029406 0.22650581 0.23271389 0.23890748 0.24509479
 0.25127687 0.25744546 0.26361337 0.26977084 0.27592073 0.28206345
 0.28820282 0.29433585 0.30046395 0.30658926 0.31270819 0.3188207
 0.32492978 0.33103075 0.33712537 0.34321485 0.34930332 0.35538923
 0.36146999 0.36754706 0.37361859 0.37968696 0.38575132 0.39181134
 0.39786884 0.40392488 0.40997774 0.41602937 0.42207959 0.42812892
 0.43417651 0.44022316 0.44626774 0.4523122  0.45835545 0.46439757
 0.47043918 0.4764794  0.48251881 0.48855644 0.49459389 0.50063092
 0.50666573 0.51269987 0.51873314 0.52476541 0.53079747 0.53682868
 0.54285867 0.54888812 0.55491635 0.56094425 0.5669717  0.57299715
 0.57902072 0.58504361 0.59106399 0.597084   0.60310283 0.60911